In [29]:
class NN_Model:

    def __init__(self, sess, name, learning_rate = 0.01):
        self.sess = sess
        self.name = name
        self._build_net()
        self.learning_rate = learning_rate

    def _build_net(self):
        with tf.device('/gpu:0'):
            with tf.variable_scope(self.name):
                # dropout (keep_prob) rate  0.7~0.5 on training, but should be 1
                # for testing
                self.keep_prob = tf.placeholder(tf.float32)
                nb_classes=7
                # input place holders
                self.X = tf.placeholder(tf.float32, [None, X_train.shape[1]])
                self.Y = tf.placeholder(tf.float32, [None, 7])

                W1 = tf.get_variable("W_1", shape =[X_train.shape[1], 200], initializer=tf.contrib.layers.xavier_initializer())            
                b1 = tf.Variable(tf.random_normal([200]), name = 'bias1')
                L1 = tf.nn.relu(tf.matmul(self.X, W1) + b1)
                L1 = tf.nn.dropout(L1, keep_prob=self.keep_prob)

                W2 = tf.get_variable("W_2", shape =[200, 400], initializer=tf.contrib.layers.xavier_initializer())

                b2 = tf.Variable(tf.random_normal([400]), name = 'bias2')
                L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
                L2 = tf.nn.dropout(L2, keep_prob=self.keep_prob)

                W3 = tf.get_variable("W_3", shape =[400, 400], initializer=tf.contrib.layers.xavier_initializer())

                b3 = tf.Variable(tf.random_normal([400]), name = 'bias3')
                L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
                L3 = tf.nn.dropout(L3, keep_prob=self.keep_prob)

                W4 = tf.get_variable("W_4", shape =[400, 200], initializer=tf.contrib.layers.xavier_initializer())
                b4 = tf.Variable(tf.random_normal([200]), name = 'bias4')
                L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
                L4 = tf.nn.dropout(L4, keep_prob=self.keep_prob)

                W5 = tf.get_variable("W_5", shape =[200, nb_classes], initializer=tf.contrib.layers.xavier_initializer())

                b5 = tf.Variable(tf.random_normal([nb_classes]), name = 'bias4')
                self.logits = tf.matmul(L4, W5) + b5
                '''
                Tensor("add_1:0", shape=(?, 10), dtype=float32)
                '''

        # define cost/loss & optimizer
        with tf.device('/cpu:0'):
            self.cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=self.logits, labels=self.Y))
            self.optimizer = tf.train.AdamOptimizer(learning_rate= 0.001).minimize(self.cost)
            self.predict_proba_ = tf.nn.softmax(self.logits)
            prediction = tf.argmax(self.logits, 1)
            correct_prediction = tf.equal(prediction, tf.argmax(self.Y, 1))

            self.accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    def predict(self, x_test, keep_prop=1.0):
        return self.sess.run(self.logits, feed_dict={self.X: x_test, self.keep_prob: keep_prop})

    def get_accuracy(self, x_test, y_test, keep_prop=1.0):
        return self.sess.run(self.accuracy, feed_dict={self.X: x_test, self.Y: y_test, self.keep_prob: keep_prop})
    
    def fit(self, x_data, y_data, keep_prop=0.7):       
        self.sess.run(tf.global_variables_initializer())
        tf.reset_default_graph()
        for step in range(2000):
            cost, _ = self.sess.run([self.cost, self.optimizer], feed_dict={self.X: x_data, self.Y: y_data, self.keep_prob: keep_prop})
        print('train finished')
        return self
        


In [30]:
score_list = {}
for name, ratio in zip(feature_select_names, feature_select):
    print(name)
    cv= KFold(5, shuffle = True, random_state = 42).split(standard_data)
    acc=[]
    for i, data in enumerate(cv):
        X_train = standard_data.iloc[data[0]]
        X_test = standard_data.iloc[data[1]]
        y_train = train_target_onehot[data[0]]
        y_test = train_target_onehot[data[1]]
        tf.reset_default_graph()
        # gpu 메모리할당
        gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
        sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
        nn = NN_Model(sess, 'nn'+ name +str(i))
        nn.fit(X_train, y_train)
        acc.append(nn.get_accuracy(X_test, y_test))
    print('score : {:.4f}({:.4f})'.format(np.array(acc).mean(), np.array(acc).std()))
    print('-' * 40)

all
train finished
train finished
train finished
train finished
train finished
score : 0.8667(0.0084)
----------------------------------------
3/4
train finished
train finished
train finished
train finished
train finished
score : 0.8693(0.0101)
----------------------------------------
half
train finished
train finished
train finished
train finished
train finished
score : 0.8700(0.0084)
----------------------------------------
1/4
train finished
train finished
train finished
train finished
train finished
score : 0.8681(0.0082)
----------------------------------------
